<a href="https://colab.research.google.com/github/krishnakalisarkar/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [26.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [510 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,420 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,263 kB]
Get:13 http://ppa.launchpad.net/graphics-dri

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-05 20:33:41--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-08-05 20:33:41 (6.47 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

#Load Amazon Data into Spark DataFrame


In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

##Deliverable 2: Determine Bias of Vine Reviews.

In [5]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28HBXXO1UEVJT|          5|            0|          0|   N|                N|
| RZKRFS2UUMFFU|          5|           15|         15|   N|                Y|
|R2WAU9MD9K6JQA|          3|            6|          8|   N|                N|
|R36SCTKYTVPZPC|          5|           10|         11|   N|                Y|
|R10BM6JUOJX27Q|          3|            0|          0|   Y|                N|
| RCLZ5OKZNUSY4|          5|            0|          0|   N|                Y|
|R1S65DJYEI89G4|          4|            8|         17|   N|                N|
|R3KQYBQOLYDETV|          4|            2|          2|   N|                N|
|R3QV8K7CSU8K2W|          5|            0|          0|   N|                N|
|R3W5A1WUGO5VQ0|          4|            0|          1|   N|     

In [6]:
# Step 1: Total number of votes > 20
top_reviews_df = vine_df.filter("total_votes>=20")
top_reviews_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R18VIM840CEFRP|          1|           16|        105|   N|                N|
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
| RBQZC5A3TSWT5|          5|           11|         22|   N|                Y|
| RW00TDPV9U93E|          1|           35|         76|   N|                N|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|
|R18I0XBYWFSR5T|          1|           16|         50|   N|                N|
|R1VSQ3QK30VDI0|          5|           23|         35|   N|                N|
|R3NEJBF4FXMVUU|          5|           87|         90|   N|     

In [7]:
# Step 2: Calculating percentage in 2 steps.
percentage_df = top_reviews_df.withColumn('percentage',top_reviews_df['helpful_votes']/top_reviews_df["total_votes"])
Percentage_df = percentage_df.withColumn("Percentage",percentage_df["percentage"]*100)
Percentage_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|        Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R18VIM840CEFRP|          1|           16|        105|   N|                N|15.238095238095239|
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y| 95.07042253521126|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y| 95.36082474226805|
| RBQZC5A3TSWT5|          5|           11|         22|   N|                Y|              50.0|
| RW00TDPV9U93E|          1|           35|         76|   N|                N| 46.05263157894737|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N| 81.81818181818183|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N| 88.23529411764706|
|R18I0XBYWFSR5T|          1|  

In [8]:
# Filtering percentage above 50%
high_percent_df = Percentage_df.filter("Percentage > 50")
high_percent_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|95.07042253521126|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|95.36082474226805|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|81.81818181818183|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|88.23529411764706|
|R1VSQ3QK30VDI0|          5|           23|         35|   N|                N|65.71428571428571|
|R3NEJBF4FXMVUU|          5|           87|         90|   N|                Y|96.66666666666667|
|R34ZTQTHWRHT5R|          5|           18|         20|   N|                Y|             90.0|
|R367X555006ROC|          1|           3

In [9]:
# Step 3: Filter vine(paid)= Y
vine_paid_df = high_percent_df.filter(high_percent_df["vine"] == "Y")
vine_paid_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R2Z8083V8ZFQVZ|          3|           82|         87|   Y|                N|94.25287356321839|
| RK25TAO4GGS5G|          3|           22|         24|   Y|                N|91.66666666666666|
|R2SJQG3C6KY0M8|          3|           18|         20|   Y|                N|             90.0|
|R12CHC0CB2WASU|          5|           43|         48|   Y|                N|89.58333333333334|
|R2RY328TIDXMTE|          2|           50|         61|   Y|                N| 81.9672131147541|
|R3K8OQU4PEP8W7|          4|           40|         46|   Y|                N|86.95652173913044|
|R1JI1A7JT99538|          3|           28|         40|   Y|                N|             70.0|
|R247K3K5SVPJUL|          3|           1

In [10]:
# Step 4: Filter vine(unpaid)= N
vine_unpaid_df = high_percent_df.filter(high_percent_df["vine"] == "N")
vine_unpaid_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|95.07042253521126|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|95.36082474226805|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|81.81818181818183|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|88.23529411764706|
|R1VSQ3QK30VDI0|          5|           23|         35|   N|                N|65.71428571428571|
|R3NEJBF4FXMVUU|          5|           87|         90|   N|                Y|96.66666666666667|
|R34ZTQTHWRHT5R|          5|           18|         20|   N|                Y|             90.0|
|R367X555006ROC|          1|           3

In [11]:
# Step 5:  Total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for review(paid).

##Calculations

###Paid Reviews

In [13]:
from pyspark.sql.functions import count
# Total number of reviews
total_reviews_count = vine_paid_df.count()
total_reviews_count

4743

In [14]:
# Total number of 5 star paid reviews
five_star_paid_df= vine_paid_df.filter("star_rating == 5")
five_star_paid_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R12CHC0CB2WASU|          5|           43|         48|   Y|                N|89.58333333333334|
|R27UEUDUYPLXZC|          5|           20|         24|   Y|                N|83.33333333333334|
|R2QQDNBYDIEL9F|          5|           35|         41|   Y|                N|85.36585365853658|
|R3FO8YQ2BKL1RG|          5|           93|         94|   Y|                N|98.93617021276596|
|R3DRE8T4NTEWNV|          5|           28|         28|   Y|                N|            100.0|
|R2ZD6GIDCKAJNM|          5|          118|        139|   Y|                N|84.89208633093526|
| RHGNFBA3HATNJ|          5|          223|        237|   Y|                N|94.09282700421942|
|R3IRTK29MZLYTQ|          5|           2

In [15]:
# Total Five star paid reviews
total_five_star_paid_count = five_star_paid_df.count()
total_five_star_paid_count

1604

In [16]:
# Percentage of 5 star paid reviews
paid_five_star_percentage = (total_five_star_paid_count/total_reviews_count)*100
round(paid_five_star_percentage, 3)

33.818

###Unpaid Reviews

In [17]:
# Total number of unpaid reviews
total_reviews_count_unpaid = vine_unpaid_df.count()
total_reviews_count_unpaid

328313

In [18]:
# Total number of 5 star unpaid reviews
five_star_unpaid_df= vine_unpaid_df.filter("star_rating == 5")
five_star_unpaid_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|95.36082474226805|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|88.23529411764706|
|R1VSQ3QK30VDI0|          5|           23|         35|   N|                N|65.71428571428571|
|R3NEJBF4FXMVUU|          5|           87|         90|   N|                Y|96.66666666666667|
|R34ZTQTHWRHT5R|          5|           18|         20|   N|                Y|             90.0|
| RSD6TF0E24D9P|          5|           30|         34|   N|                Y|88.23529411764706|
|R2DLFW0GMIFTKP|          5|           53|         58|   N|                Y|91.37931034482759|
|R2DK25C3UOICDC|          5|           1

In [19]:
# Total Five star unpaid reviews
total_five_star_unpaid_count = five_star_unpaid_df.count()
total_five_star_unpaid_count

167855

In [20]:
# Percentage of 5 star unpaid reviews
unpaid_five_star_percentage = (total_five_star_unpaid_count/total_reviews_count_unpaid)*100
round(unpaid_five_star_percentage, 3)

51.127

#Summary

In [28]:
print(f" The total number of paid vine reviewers are {total_reviews_count}")
print(f" The total number of unpaid vine reviewers are {total_reviews_count_unpaid}")

 The total number of paid vine reviewers are 4743
 The total number of unpaid vine reviewers are 328313


In [27]:
print(f" The total number of paid vine reviewers with 5 star reviews are {total_five_star_paid_count}")
print(f" The number of unpaid vine reviewers with 5 star reviews are {total_five_star_unpaid_count}")

 The total number of paid vine reviewers with 5 star reviews are 1604
 The number of unpaid vine reviewers with 5 star reviews are 167855


In [25]:
print(f" The percentage of paid vine reviewers with 5 star reviews are {round(paid_five_star_percentage,3)} % ")
print(f" The percentage of unpaid vine reviewers with 5 star reviews are {round(unpaid_five_star_percentage,3)} % ")

 The percentage of paid vine reviewers with 5 star reviews are 33.818 % 
 The percentage of unpaid vine reviewers with 5 star reviews are 51.127 % 


## End of Deliverable 2